# 🎓 DSPy ReAct Agents - Fundamentos (Abordagem Linear)

**Versão:** 1.0 - Basic Linear  
**Nível:** Iniciante  
**Tempo estimado:** 20-25 minutos  
**Abordagem:** Conceitos → Construção → Execução

---

## 📋 Sobre este Notebook

Este notebook oferece uma **introdução completa e progressiva** aos agentes ReAct em DSPy.

Você vai aprender:
- ✅ O que são agentes ReAct e por que usar DSPy
- ✅ Como modelar dados e criar ferramentas para agentes
- ✅ Como construir e configurar um agente ReAct do zero
- ✅ Como testar e entender o comportamento do agente

### 🎯 Pré-requisitos
- Python básico (funções, classes, tipos)
- Conceitos básicos de LLMs
- Familiaridade com Jupyter Notebooks

### 📚 Navegação entre Notebooks

**Série DSPy ReAct Agents:**
1. **→ Você está aqui:** Fundamentos (Linear)
2. [Fundamentos (Hands-On)](dspy_agents_basic_handson_final.ipynb) - Mesmos conceitos, abordagem prática
3. [Otimização Avançada (Linear)](dspy_agents_advanced_linear_final.ipynb) - Melhorando performance
4. [Otimização Avançada (Hands-On)](dspy_agents_advanced_handson_final.ipynb) - Otimização na prática

---

Based on: https://dspy.ai/tutorials/customer_service_agent/

## Setup and Imports

In [85]:
import dspy
import os
from datetime import datetime, timedelta
from typing import List, Optional, Dict, Any
from pydantic import BaseModel, Field
import json
import uuid
from dotenv import load_dotenv
load_dotenv()

True

In [86]:
from langfuse import get_client
 
langfuse = get_client()
 
# Verify connection
if langfuse.auth_check():
    print("Langfuse client is authenticated and ready!")
else:
    print("Authentication failed. Please check your credentials and host.")

Langfuse client is authenticated and ready!


## Enable Tracing for DSPy

In [87]:
from openinference.instrumentation.dspy import DSPyInstrumentor
DSPyInstrumentor().instrument()

Attempting to instrument while already instrumented


## Data Models

Define Pydantic models for the data structures we'll use.

In [88]:
class Date(BaseModel):
    year: int
    month: int
    day: int
    
    def __str__(self):
        return f"{self.year:04d}-{self.month:02d}-{self.day:02d}"

class UserProfile(BaseModel):
    name: str
    user_id: str
    email: str
    phone: str
    frequent_flyer_number: Optional[str] = None

class Flight(BaseModel):
    flight_id: str
    flight_number: str
    departure_airport: str
    arrival_airport: str
    departure_time: str
    arrival_time: str
    duration_minutes: int
    price: float
    available_seats: int
    
class Itinerary(BaseModel):
    itinerary_id: str
    user_id: str
    flights: List[Flight]
    total_price: float
    booking_date: str
    status: str  # "confirmed", "cancelled", "pending"

class Ticket(BaseModel):
    ticket_id: str
    user_id: str
    itinerary_id: str
    confirmation_number: str
    issue_description: Optional[str] = None
    status: str  # "active", "resolved", "pending"

## Mock Database Setup

Create dummy databases for users, flights, itineraries, and tickets.

In [89]:
# Mock databases
users_db = {
    "Adam": UserProfile(
        name="Adam",
        user_id="user_001",
        email="adam@example.com",
        phone="+1-555-0101",
        frequent_flyer_number="FF12345"
    ),
    "Sarah": UserProfile(
        name="Sarah",
        user_id="user_002",
        email="sarah@example.com",
        phone="+1-555-0102"
    )
}

flights_db = {
    "SFO-JFK": [
        Flight(
            flight_id="f001",
            flight_number="AA101",
            departure_airport="SFO",
            arrival_airport="JFK",
            departure_time="08:00",
            arrival_time="16:30",
            duration_minutes=330,
            price=450.00,
            available_seats=15
        ),
        Flight(
            flight_id="f002",
            flight_number="UA205",
            departure_airport="SFO",
            arrival_airport="JFK",
            departure_time="14:00",
            arrival_time="22:45",
            duration_minutes=345,
            price=380.00,
            available_seats=8
        )
    ],
    "JFK-LAX": [
        Flight(
            flight_id="f003",
            flight_number="DL302",
            departure_airport="JFK",
            arrival_airport="LAX",
            departure_time="10:00",
            arrival_time="13:30",
            duration_minutes=390,
            price=520.00,
            available_seats=12
        )
    ]
}

itineraries_db = {}
tickets_db = {}

## Tool Functions

Define the tools that our agent will use to interact with the airline system.

In [90]:
def fetch_flight_info(departure: str, arrival: str, date: str) -> str:
    """
    Fetch available flights for a specific route and date.
    
    Args:
        departure: Departure airport code (e.g., 'SFO')
        arrival: Arrival airport code (e.g., 'JFK')
        date: Flight date in YYYY-MM-DD format
    
    Returns:
        JSON string with available flights
    """
    route = f"{departure}-{arrival}"
    flights = flights_db.get(route, [])
    
    if not flights:
        return json.dumps({"error": f"No flights found for route {route} on {date}"})
    
    flights_data = [flight.model_dump() for flight in flights]
    return json.dumps({"flights": flights_data, "count": len(flights_data)})

def pick_flight(departure: str, arrival: str, preference: str = "duration") -> str:
    """
    Pick the best flight based on user preference.
    
    Args:
        departure: Departure airport code
        arrival: Arrival airport code
        preference: 'duration' for shortest flight, 'price' for cheapest
    
    Returns:
        JSON string with selected flight
    """
    route = f"{departure}-{arrival}"
    flights = flights_db.get(route, [])
    
    if not flights:
        return json.dumps({"error": f"No flights available for route {route}"})
    
    if preference == "duration":
        best_flight = min(flights, key=lambda f: f.duration_minutes)
    else:  # price
        best_flight = min(flights, key=lambda f: f.price)
    
    return json.dumps({"selected_flight": best_flight.model_dump(), "reason": f"Best {preference}"})

def get_user_info(name: str) -> str:
    """
    Retrieve user profile information.
    
    Args:
        name: User's name
    
    Returns:
        JSON string with user profile
    """
    user = users_db.get(name)
    if not user:
        return json.dumps({"error": f"User {name} not found"})
    
    return json.dumps({"user": user.model_dump()})

def book_flight(user_name: str, flight_id: str, date: str) -> str:
    """
    Book a flight for a user.
    
    Args:
        user_name: Name of the user
        flight_id: ID of the flight to book
        date: Travel date
    
    Returns:
        JSON string with booking confirmation
    """
    user = users_db.get(user_name)
    if not user:
        return json.dumps({"error": f"User {user_name} not found"})
    
    # Find the flight
    flight = None
    for route_flights in flights_db.values():
        for f in route_flights:
            if f.flight_id == flight_id:
                flight = f
                break
        if flight:
            break
    
    if not flight:
        return json.dumps({"error": f"Flight {flight_id} not found"})
    
    if flight.available_seats <= 0:
        return json.dumps({"error": "No available seats"})
    
    # Create itinerary
    itinerary_id = str(uuid.uuid4())
    confirmation_number = f"CONF{uuid.uuid4().hex[:8].upper()}"
    
    itinerary = Itinerary(
        itinerary_id=itinerary_id,
        user_id=user.user_id,
        flights=[flight],
        total_price=flight.price,
        booking_date=datetime.now().strftime("%Y-%m-%d"),
        status="confirmed"
    )
    
    itineraries_db[itinerary_id] = itinerary
    
    # Update available seats
    flight.available_seats -= 1
    
    return json.dumps({
        "success": True,
        "confirmation_number": confirmation_number,
        "itinerary_id": itinerary_id,
        "flight": flight.model_dump(),
        "total_price": flight.price,
        "message": f"Flight {flight.flight_number} booked successfully for {user_name}"
    })

def cancel_itinerary(itinerary_id: str) -> str:
    """
    Cancel an existing itinerary.
    
    Args:
        itinerary_id: ID of the itinerary to cancel
    
    Returns:
        JSON string with cancellation result
    """
    itinerary = itineraries_db.get(itinerary_id)
    if not itinerary:
        return json.dumps({"error": f"Itinerary {itinerary_id} not found"})
    
    if itinerary.status == "cancelled":
        return json.dumps({"error": "Itinerary already cancelled"})
    
    itinerary.status = "cancelled"
    
    # Restore available seats
    for flight in itinerary.flights:
        flight.available_seats += 1
    
    return json.dumps({
        "success": True,
        "message": f"Itinerary {itinerary_id} cancelled successfully"
    })

def file_ticket(user_name: str, issue_description: str) -> str:
    """
    File a customer support ticket.
    
    Args:
        user_name: Name of the user filing the ticket
        issue_description: Description of the issue
    
    Returns:
        JSON string with ticket information
    """
    user = users_db.get(user_name)
    if not user:
        return json.dumps({"error": f"User {user_name} not found"})
    
    ticket_id = str(uuid.uuid4())
    
    ticket = Ticket(
        ticket_id=ticket_id,
        user_id=user.user_id,
        itinerary_id="",  # May not be related to specific itinerary
        confirmation_number="",
        issue_description=issue_description,
        status="pending"
    )
    
    tickets_db[ticket_id] = ticket
    
    return json.dumps({
        "success": True,
        "ticket_id": ticket_id,
        "message": f"Support ticket filed successfully. Ticket ID: {ticket_id}"
    })

## DSPy Configuration

Set up the language model and configure DSPy.

In [91]:
# Configure OpenAI API key (set this in your environment)
# os.environ["OPENAI_API_KEY"] = "your-api-key-here"

# Initialize the language model
# lm = dspy.LM('openai/gpt-4o-mini')
lm = dspy.LM('groq/openai/gpt-oss-120b')
dspy.configure(lm=lm)

## Customer Service Agent Signature

Define the DSPy signature for our customer service agent.

In [92]:
class DSPyAirlineCustomerService(dspy.Signature):
    """
    You are an airline customer service agent that helps users book and manage flights.
    You are given a list of tools to handle user requests, and you should decide the right tool to use
    in order to fulfill users' requests.
    
    Available tools:
    - fetch_flight_info: Get available flights for a route and date
    - pick_flight: Select the best flight based on duration or price
    - get_user_info: Retrieve user profile information
    - book_flight: Book a flight for a user
    - cancel_itinerary: Cancel an existing booking
    - file_ticket: File a customer support ticket
    
    Always be helpful, professional, and provide clear information about flights and bookings.
    """
    
    user_request = dspy.InputField(desc="The user's request or question")
    response = dspy.OutputField(desc="Your response to the user")

## Create the ReAct Agent

Initialize the DSPy ReAct agent with our tools.

In [93]:
# Define the tools for the agent
tools = [
    fetch_flight_info,
    pick_flight,
    get_user_info,
    book_flight,
    cancel_itinerary,
    file_ticket
]

# Create the ReAct agent
agent = dspy.ReAct(
    signature=DSPyAirlineCustomerService,
    tools=tools,
    max_iters=10
)

---

# 🎯 Próximos Passos

## O que aprendemos

Neste notebook, você aprendeu:

✅ O que são agentes ReAct e como funcionam
✅ Como modelar dados com Pydantic
✅ Como criar ferramentas (tools) para agentes
✅ Como configurar e criar um agente ReAct com DSPy
✅ Como testar o agente com diferentes casos de uso

## Para onde ir agora?

### Opção 1: Abordagem Hands-On
Se você prefere aprender fazendo, veja o notebook [Fundamentos (Hands-On)](dspy_agents_basic_handson_full.ipynb).

### Opção 2: Otimização Avançada
Pronto para melhorar seu agente? Veja:
- [Otimização Avançada (Linear)](dspy_agents_advanced_linear_full.ipynb) - Para entender a teoria
- [Otimização Avançada (Hands-On)](dspy_agents_advanced_handson_full.ipynb) - Para praticar direto

### Recursos Adicionais
- [Documentação oficial do DSPy](https://dspy.ai)
- [Tutorial original](https://dspy.ai/tutorials/customer_service_agent/)
- [Paper ReAct](https://arxiv.org/abs/2210.03629)
